In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests

## Imports

In [2]:
from web3 import Web3

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# To load ABI
import json

# For Http calls
import requests

# For regular expressions
import re

# To decode base64
import base64

# Display HTML contents
from IPython import display

## Constants

In [3]:
# Uniswap v3 NFT Manager
NFT_POSITION_MANAGER = '0xC36442b4a4522E871399CD717aBDD847Ab11FE88'

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

## Load environment variables

In [4]:
dotenv_path = Path('.env/nft_amounts')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')

## Method to get Uniswap v3 factory

In [5]:
def get_abi(address:str) -> str:
    """ Returns the ABI for given address
        
    Returns:
    str
        The factory ABI or None for errors
    """
    # Get ABI of contract; use etherscan API for verifiable smart contracts ABI
    params = {'module':'contract', 'action':'getabi', 'address':{address}, 'apikey':{ETHERSCAN_API_KEY}}
    response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
    if response['status'] == '1':
        return json.loads(response['result'])
    return None

## NFT Contract

In [6]:
# Instantiate web3 instance for us to interact with the chain
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))

# NFT ABI
nft_abi = get_abi(address=NFT_POSITION_MANAGER)
assert nft_abi != None, 'NFT Position Manager ABI not available'

# Create nft contract
nft_contract = web3.eth.contract(address=NFT_POSITION_MANAGER, abi=nft_abi)
assert nft_contract != None, 'NFT contract does not exist'

In [7]:
# Token ID of interest
token_id = 952381

token_metadata = nft_contract.functions.tokenURI(tokenId=token_id).call()
assert token_metadata != None, 'Token metadata must exist'

token_metadata[:70]

'data:application/json;base64,eyJuYW1lIjoiVW5pc3dhcCAtIDElIC0gQkVSUlkvV'

In [8]:
# Decode the metadata
decoded_metadata = base64.b64decode(re.sub(r'^data:\w+\/\w+;base64,', '', token_metadata))
json_obj = json.loads(decoded_metadata.decode('utf-8'))

# We should have these metadata values
assert set(json_obj.keys()) == {'name', 'description', 'image'}, 'Missing keys'
name = json_obj['name']
description = json_obj['description']
image = json_obj['image']

In [9]:
decoded_metadata[570:650]

b' "image": "data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjkwIiBoZWlnaHQ9IjUwMCIgdml'

## Display Metadata

In [10]:
# name
# description
# display.HTML(f'{description}')
display.HTML(f'<span style="white-space: pre-line">{description}</span>')

In [11]:
# image[:70]
# Extract SVG base64 data
svg_base64 = re.sub(r'^data:\w+\/\w+\+\w+;base64,', '', image)
# svg_base64[:60]
display.HTML(f'<img src="data:image/svg+xml;base64,{svg_base64}"/>')

## Let's put together

In [12]:
display.HTML(f'''
    <h2>{name}</h2>
    <span style="white-space: pre-line">{description}</span>
    <p></p>
    <img src="data:image/svg+xml;base64,{svg_base64}"/>
    ''')

## Smaller Image

In [13]:
display.HTML(f'<img src="data:image/svg+xml;base64,{svg_base64}" width="40" height="60"/>')